<a href="https://colab.research.google.com/github/lhwong/airbnb-analytics/blob/main/paidout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
from google.colab import files
import io

uploaded = files.upload()


df2 = pd.read_csv(io.BytesIO(uploaded['airbnb_tax_01_2023-07_2023.csv']), parse_dates=['Date', 'Start Date'])

df2.info()

df2.loc[df2["Listing"] == "Level35 Ceylonz Suites overlooking KLCC, KL Tower", "Listing"] = 'R'
df2.loc[df2["Listing"] == "Level 35 Ceylonz Suites overlooking KLCC, KL Tower", "Listing"] = 'L'



result = df2.loc[df2['Type'] == "Reservation", 'Amount'].sum()

print("Total Paid Out Amount: ",result)

result = df2.loc[(df2['Type'] == "Reservation") & (df2['Listing'] == "L"), 'Amount'].sum()

print("Total Paid Out Amount for L: ",result)


result = df2.loc[(df2['Type'] == "Reservation") & (df2['Listing'] == "R"), 'Amount'].sum()

print("Total Paid Out Amount for R: ",result)



#df2.groupby(df2.Date.dt.year)['Amount'].sum()



Saving airbnb_tax_01_2023-07_2023.csv to airbnb_tax_01_2023-07_2023.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               72 non-null     datetime64[ns]
 1   Type               72 non-null     object        
 2   Confirmation Code  72 non-null     object        
 3   Start Date         72 non-null     datetime64[ns]
 4   Nights             72 non-null     int64         
 5   Guest              72 non-null     object        
 6   Listing            72 non-null     object        
 7   Details            4 non-null      object        
 8   Reference          0 non-null      float64       
 9   Currency           72 non-null     object        
 10  Amount             72 non-null     float64       
 11  Paid Out           0 non-null      float64       
 12  Host Fee           68 non-null     float64       
